In [24]:
# Import Data
## Conventional Generators data
Conventional_generators = [
        1 2 7 13 15 15 16 18 21 22 23 23;
        13.32 13.32 20.7 20.93 26.11 10.52 10.52 6.02 5.47 7 10.52 10.89;
        1.68 1.68 3.30 4.07 1.89 5.48 5.48 4.98 5.53 8.00 3.45 5.11;
        2.32 2.32 4.67 3.93 3.11 3.52 3.52 5.02 4.97 6.00 2.52 2.89;
        106.4 106.4 245 413.7 42 108.5 108.5 280 280 210 217 245;
        48 48 84 216 42 36 36 60 60 48 72 48;
        48 48 84 216 42 36 36 60 60 48 72 48;
    ]

##  Wind Farms data
Wind_farms = [
    3 5 16 21;
    500 500 300 300;
    120.54 115.52 53.34 38.16;
]

## Demands data
Demands = [
    1   2   3   4   5   6 7 8 9 10 13 14 15 16 18 19 20;
    84  75  139 58  55  106 97  132 135 150 205 150 245 77  258 141 100;
    500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500;
    ]

## Transmission_lines Data
Transmission_fir = [
    1 1 1 2 2 3 3 4 5 6 7 8 8 9 9 10 10;
    2 3 5 4 6 9 24 9 10 10 8 9 10 11 12 11 12;
    0.0146 0.2253 0.0907 0.1356 0.205 0.1271 0.084 0.111 0.094 0.0642 0.0652 0.1762 0.1762 0.084 0.084 0.084 0.084;
    175 175 350 175 175 175 400 175 350 175 350 175 175 400 400 400 400;
]
Transmission_sec =[
    11 11 12 12 13 14 15 15 15 16 16 17 17 18 19 20 21;
    13 14 13 23 23 16 16 21 24 17 19 18 22 21 20 23 22;
    0.0488 0.0426 0.0488 0.0985 0.0884 0.0594 0.0172 0.0249 0.0529 0.0263 0.0234 0.0143 0.1069 0.0132 0.0203 0.0112 0.0692;
    500 500 500 500 250 250 500 400 500 500 500 500 500 1000 1000 1000 500;
]
Transmission_lines = hcat(Transmission_fir, Transmission_sec)

# Get number of elements connected to the grid from Data

N_G = Int64.(size(Conventional_generators,2))       # number of Conventional_generators
N_W = Int64.(size(Wind_farms,2))                    # number of Wind_farms
N_D = Int64.(size(Demands,2))                               # number of Demands
N_l = Int64.(size(Transmission_lines,2) )                   # number of Transmission_lines
N_n = Int64.(findmax(Transmission_lines[1:2,:])[1]) # number of nodes
N_a = 2                                             # number of zones

##

# conventional generators
cg_node = Int64.(Conventional_generators[1,:])          # Node location of conventional generators [Node]
cg_production_cost = Conventional_generators[2,:]       # Production cost of conventional generators [$/MWh]
cg_upward_reserve_cost = Conventional_generators[3,:]   # Upward reserve cost of conventional generators [$/MWh]
cg_downward_reserve_cost = Conventional_generators[4,:] # Downward reserve cost of conventional generators [$/MWh]
cg_capacity = Conventional_generators[5,:]              # [MW]
cg_max_upward_reserve_provision_capability = Conventional_generators[6,:]   # [MW]
cg_max_downward_reserve_provision_capability = Conventional_generators[7,:] # [MW]

## Susceptance of lines
Susceptance = 1 ./ Transmission_lines[3,:]

Production_cost = Conventional_generators[2,:]

P̅ᴳ = Conventional_generators[5,1:end] # capacity of conventional generators
P̲ᴳ = zeros(N_G)

P̅ᵂ = Wind_farms[3,:]
P̲ᵂ = zeros(N_W)

P_dem_max = Demands[2,:]
P_dem_min = zeros(N_D);

##
U = (Demands[2,:] + ones(N_D) *45)'

C = (Conventional_generators[5,1:end]/2)'

T = zeros(4)';

## Clear the market price with inelastic demand

\begin{aligned}
\min_{p^{G}_g,p^{W}_w} \quad & \sum_{g=1}^{N_{G}}{C_{g}p^{G}_g} + \sum_{w=1}^{N_{W}}{T_{w} p^{W}_w}\\
\textrm{s.t.} \quad & 0 \geq p^{D}_{d} \geq \bar{P^{D}_{d}} \quad \quad \forall d\\
  & 0 \geq p^{G}_{g} \geq \bar{P^{G}_{g}} \quad \quad \forall g \\
  & 0 \geq p^{W}_{w} \geq \bar{P^{W}_{w}} \quad \quad \forall w \\
  & P^D - \sum_{g=1}^{N_{G}}{p^{G}_g} - \sum_{w=1}^{N_{W}}{p^{W}_w} = 0 \quad :\lambda \\
\end{aligned}

In [27]:
using JuMP
using GLPK

pᴰ= Demands[2,:] # Demends inelastic price

vector_model = Model(GLPK.Optimizer)

@variable(vector_model, pᴳ[1:N_G])
@variable(vector_model, pᵂ[1:N_W])

@constraint(vector_model, cg_con[i=1:N_G], P̲ᴳ[i] <= pᴳ[i] <= P̅ᴳ[i])
@constraint(vector_model, wt_con[i=1:N_W], P̲ᵂ[i] <= pᵂ[i] <= P̅ᵂ[i])

@constraint(vector_model, balance_con, + sum(pᴰ) - sum(pᴳ) - sum(pᵂ) .== 0)

@objective(vector_model, Min, T*pᵂ + C*pᴳ)

#print(vector_model)
optimize!(vector_model)
@show termination_status(vector_model)
@show primal_status(vector_model)
@show dual_status(vector_model)
@show objective_value(vector_model)
@show value.(pᴳ)
@show value.(pᵂ)

MCP = abs(shadow_price(balance_con)) # Market Clearing Price



termination_status(vector_model) = MathOptInterface.OPTIMAL
primal_status(vector_model) = MathOptInterface.FEASIBLE_POINT
dual_status(vector_model) = MathOptInterface.FEASIBLE_POINT
objective_value(vector_model) = 198284.31
value.(pᴳ) = [106.4, 106.4, 245.0, 0.0, 42.0, 108.5, 108.5, 210.6400000000001, 280.0, 210.0, 217.0, 245.0]
value.(pᵂ) = [120.54, 115.52, 53.34, 38.16]


140.0

## Balancing stage

In the balancing stage, generator 7 becomes out of the system (unexpected outage). The actual
production of farms 1 and 2 is 10% lower than their day‐ahead forecast, while that of farms 3 and 4 is
25% higher than their day‐ahead forecast. All remaining 11 conventional generators (but not
demands) are potential balancing service providers. There is no other balancing service provider
(except for involuntarily load curtailment). Each conventional generator as the balancing service
provider offers the upward balancing service at a price equal to the day‐ahead price plus 10% of her
production cost. Similarly, she offers the downward balancing service (if she can depending on her
day‐ahead schedule) at a price equal to the day‐ahead price minus 12% of her production cost. Clear
the balancing market, and derive the balancing price.

In [26]:
p𝒟𝒜ᴳ = value.(pᴳ) # p\scrD\scrA\^G
p𝒟𝒜ᵂ = value.(pᵂ)
p𝒟𝒜ᴰ = value.(pᴰ)

p𝒜ᴳ = copy(value.(pᴳ))
p𝒜ᵂ = copy(value.(pᵂ))
p𝒜ᴰ = copy(value.(pᴰ))

# generator 7 becomes out of the system (unexpected outage)
p𝒜ᴳ[7] = 0

# The actual production of wind farms 1 and 2 is 10% lower than their day‐ahead forecast
p𝒜ᵂ[1:2] = copy(P̅ᵂ[1:2]) * 0.9 

# The actual production of wind farms 3 and 4 is 25% higher than their day‐ahead forecast
p𝒜ᵂ[3:4] = copy(P̅ᵂ[3:4]) * 1.25; 

# System imbalance
ΔP = sum(p𝒜ᴳ-p𝒟𝒜ᴳ) + sum(p𝒜ᵂ-p𝒟𝒜ᵂ) - sum(p𝒜ᴰ-p𝒟𝒜ᴰ)


-109.231

In [ ]:
# All remaining 11 conventional generators (but not demands) are potential balancing service providers.
# There is no other balancing service provider (except for involuntarily load curtailment).
P̅ᴳ↗ 

# Conventional generator as the balancing service provider offers the upward balancing
# service at a price equal to the day‐ahead price plus 10% of her production cost
Cᴳ↗ = C + Production_cost * 0.1 # C\nearrow

# she offers the downward balancing service (if she can depending on her day‐ahead schedule)
# at a price equal to the day‐ahead price minus 12% of her production cost
Cᴳ↘ = C - Production_cost * 0.12; #C\searrow